<h1> Part 1: Battle of the Neighborhoods </h1>

<h3> In this section, we will go over two cities of South Dakota. Sioux Falls and Rapid city. We will explore which cities have more Family restaurants and will decide that it would be a good idea to open a restaurant to whatever city has fewer restaurants. Fortunately, both cities have relatively similar population and income. </h3>

<h3> Importing the libraries </h3>

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

<h3> Foursquare IDs </h3>

In [5]:
CLIENT_ID = 'WOMCMJOL5F50YRPVEIVKNQUXQKCQBMGE2WRSYIXQXHN4WNRL' # your Foursquare ID
CLIENT_SECRET = 'NEYEAKKCAXI3NZVWS2JD3ITT2R3C44XXN4MMRTRWEJVOANNN' # your Foursquare Secret
VERSION = '20200614'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WOMCMJOL5F50YRPVEIVKNQUXQKCQBMGE2WRSYIXQXHN4WNRL
CLIENT_SECRET:NEYEAKKCAXI3NZVWS2JD3ITT2R3C44XXN4MMRTRWEJVOANNN


<h3> Downtown Sioux Falls Location </h3>

In [6]:
address = '401 E 8th St, Sioux Falls, SD'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.5477655 -96.7219582


<h3> Now we search for Family restaurants in 500 meter </h3>

In [7]:
search_query = 'Family Restaurant'
radius = 500
print(search_query + ' .... OK!')

Family Restaurant .... OK!


In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WOMCMJOL5F50YRPVEIVKNQUXQKCQBMGE2WRSYIXQXHN4WNRL&client_secret=NEYEAKKCAXI3NZVWS2JD3ITT2R3C44XXN4MMRTRWEJVOANNN&ll=43.5477655,-96.7219582&v=20200614&query=Family Restaurant&radius=500&limit=30'

In [9]:
results = requests.get(url).json()

<h3> Normalizing the Jason File for better view. </h3>

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Jacky’s Restaurant,Latin American Restaurant,702 E. 8th Street,Nesmith Ave.,43.548758,-96.718123,"[{'label': 'display', 'lat': 43.54875838202667...",328,57103,US,Sioux Falls,SD,United States,"[702 E. 8th Street (Nesmith Ave.), Sioux Falls...",4b512c35f964a520e04527e3
1,K Restaurant,New American Restaurant,401 E 8th St #128,NaN,43.547232,-96.722267,"[{'label': 'display', 'lat': 43.54723153538433...",64,57103,US,Sioux Falls,SD,United States,"[401 E 8th St #128, Sioux Falls, SD 57103, Uni...",4b7c3196f964a5207f832fe3
2,Los Paisanos Hispanic Restaurant,Mexican Restaurant,802 E 10th St,NaN,43.546790,-96.716517,"[{'label': 'display', 'lat': 43.5467902, 'lng'...",452,57103,US,Sioux Falls,SD,United States,"[802 E 10th St, Sioux Falls, SD 57103, United ...",4f9ac6b8e4b0edc56029158e
3,Golden Dragon Chinese Restaurant,Chinese Restaurant,121 N Indiana Ave,NaN,43.548340,-96.715980,"[{'label': 'display', 'lat': 43.54833999595760...",486,57103,US,Sioux Falls,SD,United States,"[121 N Indiana Ave, Sioux Falls, SD 57103, Uni...",4c81837651ada1cd1acd1310
4,Falls Landing Restaurant,Hotel Bar,200 E 8th St,at N. Phillips Ave.,43.548843,-96.724868,"[{'label': 'display', 'lat': 43.54884258268844...",263,57103,US,Sioux Falls,SD,United States,"[200 E 8th St (at N. Phillips Ave.), Sioux Fal...",4b47ba88f964a520313c26e3


<h3> Name of the restaurants. </h3>

In [10]:
dataframe_filtered.name

0                        K Restaurant
1                  Jacky’s Restaurant
2    Los Paisanos Hispanic Restaurant
3            Falls Landing Restaurant
4    Golden Dragon Chinese Restaurant
Name: name, dtype: object

<h3> Now we map all the restaurants in that location. Also, there is only 5 restaurants in Downtown Sioux Falls. </h3>

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)


# add the Chinese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<h3> Now we will analyze the ratings of these restaurants. </h3>

In [13]:
venue_id = '4b7c3196f964a5207f832fe3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('Rating of  K Restaurant is ', result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
venue_id = '4b512c35f964a520e04527e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('Rating of  Jackys"s Restaurant is ',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

    
venue_id = '4b47ba88f964a520313c26e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('Rating of  Falls Landing Restaurant is ',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
venue_id = '4c81837651ada1cd1acd1310'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('Rating of Golden Dragon Chinese Restaurant is',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

venue_id = '4f9ac6b8e4b0edc56029158e'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('Rating of  Los Paisanos Hispanic Restaurant is ',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')



Rating of  K Restaurant is  7.7
Rating of  Jackys"s Restaurant is  9.0
Rating of  Falls Landing Restaurant is  7.9
Rating of Golden Dragon Chinese Restaurant is 7.5
Rating of  Los Paisanos Hispanic Restaurant is  6.3


<h3>Now we will find the location of downtown of Rapid City </h3>

In [14]:
address1 = '512 Main St #980, Rapid City, SD'

geolocator1 = Nominatim(user_agent="foursquare_agent")
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print(latitude1, longitude1)

44.0811262 -103.2260343


<h3> We will search for Family Restaurants in downtown Rapid city </h3>

In [15]:
search_query1 = 'Family Restaurant'
radius = 500
print(search_query1 + ' .... OK!')

Family Restaurant .... OK!


In [16]:
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query1, radius, LIMIT)
url1

'https://api.foursquare.com/v2/venues/search?client_id=WOMCMJOL5F50YRPVEIVKNQUXQKCQBMGE2WRSYIXQXHN4WNRL&client_secret=NEYEAKKCAXI3NZVWS2JD3ITT2R3C44XXN4MMRTRWEJVOANNN&ll=44.0811262,-103.2260343&v=20200614&query=Family Restaurant&radius=500&limit=30'

In [17]:
results1 = requests.get(url1).json()

In [18]:
# assign relevant part of JSON to venues
venues = results1['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
#dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


In [19]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Wine Cellar Restaurant,Wine Bar,513 6th St,44.080720,-103.227206,"[{'label': 'display', 'lat': 44.08072028411001...",104,57701,US,Rapid City,SD,United States,"[513 6th St, Rapid City, SD 57701, United States]",NaN,4d8a9c2d7139b1f7c134cfd4
1,Ixtapa Mexican Restaurant,Mexican Restaurant,603 Omaha St,44.082943,-103.226795,"[{'label': 'display', 'lat': 44.08294265718444...",211,57701,US,Rapid City,SD,United States,"[603 Omaha St, Rapid City, SD 57701, United St...",NaN,4d4c536d9f79f04d574e232b
2,Tally's Silver Spoon,American Restaurant,530 6th St,44.080089,-103.227222,"[{'label': 'display', 'lat': 44.08008907276414...",149,57701,US,Rapid City,SD,United States,"[530 6th St (St. Joseph), Rapid City, SD 57701...",St. Joseph,4beec3af35dc0f478439405b
3,Unkenholz Family Dental,Dentist's Office,716 Quincy St,44.078584,-103.230635,"[{'label': 'display', 'lat': 44.078584, 'lng':...",464,57701,US,Rapid City,SD,United States,"[716 Quincy St, Rapid City, SD 57701, United S...",NaN,531632e9498e04c64e4a0bbd
4,Family Memorials - West River Locations,Cemetery,910 Main St,44.082390,-103.233084,"[{'label': 'display', 'lat': 44.0823902, 'lng'...",581,57701,US,Rapid City,SD,United States,"[910 Main St, Rapid City, SD 57701, United Sta...",NaN,58b785da2f91cb604820026b
5,"American Family Insurance - Mike Pyle Agency, Inc",Insurance Office,512 West Blvd,44.081878,-103.234206,"[{'label': 'display', 'lat': 44.081878, 'lng':...",658,57701,US,Rapid City,SD,United States,"[512 West Blvd, Rapid City, SD 57701, United S...",NaN,4f0ddb21e4b0129c736e5603
6,Enigma Restaurant,Restaurant,445 Mount Rushmore Rd,44.082134,-103.231017,"[{'label': 'display', 'lat': 44.08213365611971...",413,57701,US,Rapid City,SD,United States,"[445 Mount Rushmore Rd, Rapid City, SD 57701, ...",NaN,4b5908dff964a5206c7928e3
7,Health Source Chiropractic,Chiropractor,325 Omaha St,44.082005,-103.222679,"[{'label': 'display', 'lat': 44.082005, 'lng':...",285,57701,US,Rapid City,SD,United States,"[325 Omaha St, Rapid City, SD 57701, United St...",NaN,4d62d34f9f67f04d6e7093fb
8,Diamond Family Medicine,Doctor's Office,703 Quincy St,44.077904,-103.230296,"[{'label': 'display', 'lat': 44.077904, 'lng':...",494,57701,US,Rapid City,SD,United States,"[703 Quincy St, Rapid City, SD 57701, United S...",NaN,4e54136f22711d4da5c4b8df
9,Murphy's Pub & Grill,Pub,510 9th St,44.081710,-103.232383,"[{'label': 'display', 'lat': 44.08171044649926...",511,57701,US,Rapid City,SD,United States,"[510 9th St, Rapid City, SD 57701, United States]",NaN,4ddf1ed781dcc5e5c55fc368


<h3> Not all of the location we found aren't restaurants. There is also 5 restaurants in Rapid City, but they also have agencies and medical centers in downtown which makes this place more popular.</h3>

In [20]:
dataframe_filtered.name

0                               Wine Cellar Restaurant
1                            Ixtapa Mexican Restaurant
2                                 Tally's Silver Spoon
3                              Unkenholz Family Dental
4              Family Memorials - West River Locations
5    American Family Insurance - Mike Pyle Agency, Inc
6                                    Enigma Restaurant
7                           Health Source Chiropractic
8                              Diamond Family Medicine
9                                 Murphy's Pub & Grill
Name: name, dtype: object

<h3> Here is the map of the locations we found in downtonw.</h3>

In [21]:
venues_map1 = folium.Map(location=[latitude1, longitude1], zoom_start=13) 

for lat1, lng1, label1 in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat1, lng1],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map1)

# display map
venues_map1

<h3> The ratings of the restaurants are... </h3>

In [22]:
venue_id = '4d8a9c2d7139b1f7c134cfd4'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('The rating of Wine Celler Restaurant is ',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
venue_id = '4d4c536d9f79f04d574e232b'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('The rating of  Ixtapa Mexican Restaurant is ',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
venue_id = '4beec3af35dc0f478439405b'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('The rating of Tally"s Silver Spoon is ',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
venue_id = '4b5908dff964a5206c7928e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('The rating of Enigma Restaurant is ',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
venue_id = '4ddf1ed781dcc5e5c55fc368'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('The rating of Murphy"s Pub & Grill is ',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

The rating of Wine Celler Restaurant is  6.2
The rating of  Ixtapa Mexican Restaurant is  5.8
The rating of Tally"s Silver Spoon is  8.3
The rating of Enigma Restaurant is  5.9
The rating of Murphy"s Pub & Grill is  8.0


<h3> Conclusion: When comparing with the two locations, Rapid city has more locations in downtown but the restaurants are average and there is nothing extraordinary here. Restaurants in Sioux Falls have better rating but Sioux Falls downtown doesn't have quite as much location as Rapid City has. </h3>